# 4th Iteration of Magnum Analysis
Using analysed_4 and its equivalent downsampling

In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import xarray as xr 
import scipy.stats as stat
import collections
import sys
import os
import glob
import re
sys.path.append('/home/jleland/Coding/Projects/flopter')
import flopter.core.ivdata as iv
import flopter.core.lputils as lp
import flopter.magnum.database as ut
import flopter.magnum.utils as mgut
import flopter.core.fitters as fts
import flopter.core.constants as c

In [2]:
# Create analysed dataset metadata 

path_to_datasets = '/home/jleland/data/external/magnum/'
# path_to_datasets = '/home/jleland/data/externy/magnum/'
# path_to_analysed_datasets = 'analysed_2'
# path_to_analysed_datasets = 'analysed_3'
# path_to_analysed_datasets = 'phobos_test'
# path_to_analysed_datasets = 'test'
# path_to_analysed_datasets = 'analysed_3_downsampled'
# path_to_analysed_datasets = 'analysed_4'
path_to_analysed_datasets = 'analysed_4_downsampled'
# path_to_analysed_datasets = 'analysed_5_downsampled'
os.chdir(path_to_datasets)

In [3]:
magnum_probes = lp.MagnumProbes()

## Load adc file metadata

In [4]:
os.chdir('/home/jleland/data/external/magnum/')
# os.chdir('/home/jleland/data/externy/magnum/')
meta_data_ds = xr.open_dataset('all_meta_data.nc')
print(meta_data_ds)

<xarray.Dataset>
Dimensions:                 (shot_number: 523, ts_radial_pos: 46)
Coordinates:
  * shot_number             (shot_number) int32 0 1 2 3 4 ... 519 520 521 522
    ts_number               (shot_number) float64 ...
    ts_timestamp            (shot_number) float64 ...
    ts_time                 (shot_number) datetime64[ns] ...
    adc_index               (shot_number) float64 ...
    adc_time                (shot_number) datetime64[ns] ...
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Data variables:
    adc_filename            (shot_number) object ...
    ts_density              (shot_number, ts_radial_pos) float64 ...
    ts_temperature          (shot_number, ts_radial_pos) float64 ...
    ts_d_density            (shot_number, ts_radial_pos) float64 ...
    ts_d_temperature        (shot_number, ts_radial_pos) float64 ...
    adc_folder              (shot_number) object ...
    adc_calibration_index   (shot_number) object ...
    adc_4_

# Creation of Dataset from adc_file index selection

In [5]:
def preprocess_average(ds, dims_to_avg=('sweep', 'direction')):
    ds = ds.reset_index('time', drop=True).load()
    sweep_min, sweep_max = find_sweep_limit(ds)
    ds_trimmed = ds.sel(sweep=slice(sweep_min, sweep_max))

    ds_avg = ds_trimmed.mean(['sweep', 'direction'])
    ds_avg = ds_avg.assign({
        'd_current': ds_trimmed.std(['sweep', 'direction'])['current'] / np.sqrt(ds_avg['current'].size),
        'std_current': ds_trimmed.std(['sweep', 'direction'])['current']
    })
    return ds_avg

In [6]:
def fit_magnum_ds(magnum_subset_ds, probes=('L', 'S', 'B', 'R'), ax=True, scan_param='tilt', threshold='auto',
                  fitter=None, **kwargs):
    if fitter is None:
        fitter = fts.FullIVFitter()

    metadata_labels = [
        scan_param,
        'probe',
        'B',
        'ts_temp',
        'ts_dens',
        'fit_success_fl',
    ]
    fit_param_labels = [
        'temp',
        'd_temp',
        'isat',
        'd_isat',
        'a',
        'd_a',
        'v_f',
        'd_v_f',
        'dens',
        'd_dens',
        'chi2',
        'reduced_chi2'
    ]
    all_labels = metadata_labels + fit_param_labels
    fit_df = pd.DataFrame(columns=all_labels)

    for scan_param_value in magnum_subset_ds[scan_param].values:
        scan_param_ds = magnum_subset_ds.sel(**{scan_param: scan_param_value})
        for probe in probes:
            probe_paramscan_ds = scan_param_ds.sel(probe=probe)
            print(probe_paramscan_ds)
            probe_paramscan_ds = probe_paramscan_ds.where(np.isfinite(probe_paramscan_ds['voltage']), drop=True)
            probe_paramscan_ds = probe_paramscan_ds.where(np.isfinite(probe_paramscan_ds['current']), drop=True)

            if threshold == 'auto':
                # Auto option finds the first point the other side of the floating potential and then selects up to that
                threshold = 1

            if isinstance(threshold, int):
                iv_indices = np.where(probe_paramscan_ds.current < 0)[0]
                if iv_indices[0] == 0:
                    extreme_index = max(iv_indices)
                    extension = [extreme_index + (i + 1) for i in range(threshold)]
                    ext_iv_indices = np.concatenate((iv_indices, extension))
                else:
                    extreme_index = min(iv_indices)
                    extension = [extreme_index - (threshold - i) for i in range(threshold)]
                    ext_iv_indices = np.concatenate((extension, iv_indices))
                probe_paramscan_ds = probe_paramscan_ds.isel(time=ext_iv_indices)
            elif isinstance(threshold, float):
                probe_paramscan_ds = probe_paramscan_ds.where(probe_paramscan_ds.current < threshold, drop=True)
#             else:
#                 print('No threshold set, continuing with full sweep.')

            if scan_param == 'tilt':
                alpha = np.radians(scan_param_value)
            else:
                alpha = np.radians(probe_paramscan_ds['tilt'].values[0])

            if len(probe_paramscan_ds.time) == 0:
                print('Time has no length, continuing...')
                continue

            shot_iv = iv.IVData.from_dataset(probe_paramscan_ds)
#             shot_iv = iv.IVData(probe_paramscan_ds['voltage'].values,
#                                 -probe_paramscan_ds['current'].values,
#                                 probe_paramscan_ds['shot_time'].values,
#                                 sigma=probe_paramscan_ds['d_current'].values)

            try:
                shot_fit = shot_iv.multi_fit(sat_region=-40, **kwargs)
#                 shot_fit = fitter.fit_iv_data(shot_iv, sigma=shot_iv['sigma'])

                dens = magnum_probes[probe].get_density(np.abs(shot_fit.get_isat()), shot_fit.get_temp(), alpha=alpha)
                d_dens = magnum_probes[probe].get_d_density(
                    np.abs(shot_fit.get_isat()),
                    shot_fit.get_isat_err(),
                    shot_fit.get_temp(),
                    shot_fit.get_temp_err(),
                    alpha=alpha
                )
                if isinstance(dens, collections.Iterable):
                    dens = dens[0]
                    d_dens = d_dens[0]

                fit_params = {
                    'fit_success_fl': True,
                    'temp': shot_fit.get_temp(),
                    'd_temp': shot_fit.get_temp_err(),
                    'isat': shot_fit.get_isat(),
                    'd_isat': shot_fit.get_isat_err(),
                    'a': shot_fit.get_sheath_exp(),
                    'd_a': shot_fit.get_sheath_exp_err(),
                    'v_f': shot_fit.get_floating_pot(),
                    'd_v_f': shot_fit.get_floating_pot_err(),
                    'dens': dens,
                    'd_dens': d_dens,
                    'chi2': shot_fit.chi2,
                    'reduced_chi2': shot_fit.reduced_chi2,
                }
                if ax is not None:
                    if ax is True:
                        fig, ax = plt.subplots()
                    ax.errorbar(shot_iv['V'], shot_iv['I'], yerr=shot_iv['sigma'],
                                ecolor='silver', color='silver', marker='+', zorder=1)
                    ax.plot(*shot_fit.get_fit_plottables())
            except RuntimeError as e:
                print(f'WARNING: Failed on {scan_param}={scan_param_value} with probe {probe}')
                print(e)
                fit_params = {label: np.NaN for label in fit_param_labels}
                fit_params['fit_success_fl'] = False

            fit_df = fit_df.append({
                scan_param: scan_param_value,
                'probe': probe,
                'B': np.around(probe_paramscan_ds['shot_b_field'].mean().values, decimals=1),
                'ts_temp': probe_paramscan_ds['ts_temperature'].mean().values,
                'ts_dens': probe_paramscan_ds['ts_density'].mean().values,
                **fit_params,
            }, ignore_index=True)

    return fit_df

## Different Applications of this Dataset
This is after having been fed different indices, and then different plots constructed
This is, firstly, the anglescans



---
### Indices for different Anglescans

In [7]:
# Hydrogen Shots
# super_title = 'H shot @ 0.8T'
indices_08T_H = [41,42,43,44,45,46,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63]

# super_title = 'H shot @ 1.2T'
indices_12T_H = [320,321,322,323,324,325,327,328,329,330,331,336,337,338,339,340,341,342,343,344,345,346]

# super_title = 'H shot @ 1.5T'
indices_15T_H = [490,491,492,493,494,495,496,497,498,499,500,503,504,505,506,507,509,510,511,512,513,514]

# Helium Shots
# super_title = 'He shot @ 0.8T'
indices_08T_He = [357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386]

# super_title = 'He shot @ 1.2T'
indices_12T_He = [245,246,247,248,249,250,251,252,254,255,256,261,262,263,264,265,266,267,268,269,270,271]

# Deuterium Shot
# super_title = 'D shot @ 0.8T'
indices_08T_D = [462,463,464,465,466,467,468,469,470,471,472,475,476,477,478,479,480,481,482,483,484,485]

### Indices for different Density Scans

In [8]:
# 0.8T Scans
super_title = 'H scan @ 0.8T, tilt=10'
indices_n_e_08T_H = [124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,142,143,144,145,146,147]

super_title = 'H scan @ 0.8T, tilt=8'
indices_n_e_08T_H_8deg = [276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,301,302,303,304,305,306,307,308,309,310,311]

# Other Scans
super_title = 'H scan @ 0.8T, tilt=10'
indices_n_e_12T_H = [199,200,201,202,203,204,205,206,207,208,209,210]

super_title = 'H scan @ 0.8T, tilt=10'
indices_n_e_15T_H = [389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415]

super_title = 'H axial scan @ 0.8T, tilt=10'
axial_scan_indices = [132,148,149,150,151,152,153,154,155,156,157,158,159,160,161,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]

In [9]:
DEFAULT_SETTINGS = {'mode': 3, 'trimming_vals': (0.3, 0.02, 0.02), 'sat_region': -30}

def fit_multi_magnum_ds(angle_scan_ds, probes_settings={('S', 'L', 'B'): DEFAULT_SETTINGS}, **kwargs):
    fit_dfs = []
    for probes, settings in probes_settings.items():
        if not isinstance(probes, collections.Iterable):
            probes = tuple(probes)
        fit_df = mgut.fit_magnum_ds(angle_scan_ds, probes=probes, **settings, **kwargs)
        fit_dfs.append(fit_df)
    
    return pd.concat(fit_dfs)

## Anglescan Analysis Functions
A series of small snippets for plotting and analysing anglescan data

In [10]:
angle_scan_ds = mgut.get_dataset_from_indices(indices_15T_H, anglescan_fl=True, average_direction_fl=False,
                                              path_to_analysed_datasets=path_to_analysed_datasets, 
#                                               check_sweep_trim_fl=True,
                                              )

/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)


In [11]:
angles_scan_vars = set(angle_scan_ds.variables.keys())

In [12]:
mgut.plot_anglescan_ivs(angle_scan_ds)

In [13]:
# Test settings in this box

# fit_df = mgut.fit_magnum_ds(angle_scan_ds, probes=('L'), threshold=None, plot_fl=False,  sat_region=-65, minimise_temp_fl=False, temp_from_phi_fl=False)
# fit_df = mgut.fit_magnum_ds(angle_scan_ds, probes=('L'), threshold=None, plot_fl=False, trimming_vals=(0.3, 0.0, 0.02), mode=0, sat_region=-50)
# fit_df = mgut.fit_magnum_ds(angle_scan_ds, probes=('B'), threshold=None, plot_fl=False, trimming_vals=(0.1, 0.1, 0.02), mode=0, sat_region=-25)
# fit_df = mgut.fit_magnum_ds(angle_scan_ds, probes=('R'), threshold=None, plot_fl=False, trimming_vals=(0.3, 0.15, 0.02), mode=3, sat_region=-60)
fit_3P_df = mgut.fit_magnum_ds(angle_scan_ds, probes=('L'), threshold=None, plot_fl=False, trimming_vals=(0.3, 0.0, 0.02), mode=3, sat_region=-75, fitter=fts.SimpleIVFitter())
# fit_df = mgut.fit_magnum_ds(angle_scan_ds, probes=('L'), threshold=None, plot_fl=False, trimming_vals=(0.16, 0.16, 0.02), mode=3, sat_region=-20)
# fit_df = mgut.fit_magnum_ds(angle_scan_ds, probes=('B'), threshold=None, plot_fl=False, trimming_vals=(0.2, 0.2, 0.02), mode=2, sat_region=-60)
# fit_df = mgut.fit_magnum_ds(angle_scan_ds, probes=('B'), threshold=None,  plot_fl=False, sat_region=-60, minimise_temp_fl=False)

/home/jleland/coding/projects/flopter/flopter/core/lputils.py:203: RuntimeWarning: divide by zero encountered in double_scalars
  a = ((c_1 + (c_2 / np.tan(alpha))) / np.sqrt(np.sin(alpha))) * (lambda_D / (L + g))


In [25]:
# Use this if all the settings are the same

# fit_df = mgut.fit_magnum_ds(angle_scan_ds, probes=('S', 'L'), threshold=None, plot_fl=False, trimming_vals=(0.16, 0.0, 0.02), mode=0, sat_region=-75)


# or use this if the settings need to be different

ps = {
#     ('S'): {'mode': 0, 'trimming_vals': (0.3, 0.02, 0.02), 'sat_region': -75},
#     ('B'): {'mode': 2, 'trimming_vals': (0.2, 0.1, 0.02), 'sat_region': -65}
#     ('L'): {'mode': 3, 'trimming_vals': (0.3, 0.0, 0.02), 'sat_region': -75, 'fitter':fts.SimpleIVFitter()},
#     ('S', 'L'): {'mode': 0, 'trimming_vals': (0.16, 0.0, 0.02), 'sat_region': -75},
#     ('S', 'L'): {'sat_region': -65, 'minimise_temp_fl':False, 'temp_from_phi_fl': False}, 
#     ('B'): {'mode': 0, 'trimming_vals': (0.1, 0.1, 0.02), 'sat_region': -30}
    ('S', 'L', 'B'): {'mode': 0, 'trimming_vals': (0.3, 0.0, 0.02), 'sat_region': -50},
}

fit_df = mgut.fit_multi_magnum_ds(angle_scan_ds, probes_settings=ps, threshold=None, plot_fl=False)

/home/jleland/coding/projects/flopter/flopter/core/lputils.py:203: RuntimeWarning: divide by zero encountered in double_scalars
  a = ((c_1 + (c_2 / np.tan(alpha))) / np.sqrt(np.sin(alpha))) * (lambda_D / (L + g))
/home/jleland/coding/projects/flopter/flopter/core/lputils.py:203: RuntimeWarning: divide by zero encountered in double_scalars
  a = ((c_1 + (c_2 / np.tan(alpha))) / np.sqrt(np.sin(alpha))) * (lambda_D / (L + g))
/home/jleland/coding/projects/flopter/flopter/core/lputils.py:203: RuntimeWarning: divide by zero encountered in double_scalars
  a = ((c_1 + (c_2 / np.tan(alpha))) / np.sqrt(np.sin(alpha))) * (lambda_D / (L + g))
/home/jleland/coding/projects/flopter/flopter/core/fitters.py:246: RuntimeWarning: overflow encountered in exp
  return I_0 * (1 - np.exp(-V) + np.where(v <= v_f, (a * np.float_power(np.absolute(V), [0.75])), 0))


In [14]:
fit_3P_df.fillna(value=pd.np.nan, inplace=True)
fit_3P_df['probe'] = 'L_3P'
# fit_df

In [26]:
# Concatenate if using an additional 3-parameter comparison
fit_df = pd.concat([fit_df, fit_3P_df])

In [27]:
ds = fit_df.to_xarray().set_coords(['probe', 'tilt']).swap_dims({'index':'tilt'}).drop('index')

s_shots = ds.where(ds.probe=='S', drop=True).drop('probe')
l_shots = ds.where(ds.probe=='L', drop=True).drop('probe')
b_shots = ds.where(ds.probe=='B', drop=True).drop('probe')
r_shots = ds.where(ds.probe=='R', drop=True).drop('probe')
l_3p_shots = ds.where(ds.probe=='L_3P', drop=True).drop('probe')

probes_da = xr.DataArray(['L', 'S', 'B', 'R', 'L_3P'], dims=['probe'], name='probe')

ds_stacked = xr.concat([l_shots, s_shots, b_shots, r_shots, l_3p_shots], dim=probes_da)
# ds_stacked

ds_combined = xr.merge([angle_scan_ds, ds_stacked])
ds_combined['normal_tilt'] = 90 - ds_combined['tilt']
# ds_combined

In [28]:
import importlib
importlib.reload(mgut)

<module 'flopter.magnum.utils' from '/home/jleland/coding/projects/flopter/flopter/magnum/utils.py'>

In [29]:
ds_combined_alt = mgut.combine_fit_ds(fit_df, angle_scan_ds)
print(ds_combined)
ds_combined_alt

<xarray.Dataset>
Dimensions:                 (probe: 5, tilt: 11, time: 50, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'B' 'L' 'L_3P' 'R' 'S'
  * tilt                    (tilt) float64 0.0 1.0 2.0 3.0 ... 7.0 8.0 9.0 10.0
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Dimensions without coordinates: time
Data variables:
    voltage                 (tilt, probe, time) float64 -13.22 -14.78 ... -10.78
    current                 (tilt, probe, time) float64 0.993 0.98 ... 0.4267
    shot_time               (tilt, probe, time) float64 4.339 4.339 ... 4.293
    start_time              (tilt, probe) float64 4.337 4.484 ... 4.408 4.291
    d_current               (tilt, probe, time) float64 0.004939 ... 0.002508
    std_current             (tilt, probe, time) float64 0.1801 ... 0.09084
    ts_density              (probe, tilt, ts_radial_pos) float64 nan nan ... nan
    ts_temperature          (probe, tilt, ts_radial_pos) float6

<xarray.Dataset>
Dimensions:                 (probe: 4, tilt: 11, time: 50, ts_radial_pos: 46)
Coordinates:
  * probe                   (probe) object 'B' 'L' 'R' 'S'
  * tilt                    (tilt) float64 0.0 1.0 2.0 3.0 ... 7.0 8.0 9.0 10.0
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
Dimensions without coordinates: time
Data variables:
    voltage                 (tilt, probe, time) float64 -13.22 -14.78 ... -10.78
    current                 (tilt, probe, time) float64 0.993 0.98 ... 0.4267
    shot_time               (tilt, probe, time) float64 4.339 4.339 ... 4.293
    start_time              (tilt, probe) float64 4.337 4.484 ... 4.408 4.291
    d_current               (tilt, probe, time) float64 0.004939 ... 0.002508
    std_current             (tilt, probe, time) float64 0.1801 ... 0.09084
    ts_density              (probe, tilt, ts_radial_pos) float64 nan nan ... nan
    ts_temperature          (probe, tilt, ts_radial_pos) float64 nan nan ... nan
    ts_d_density            (probe, tilt, ts_radial_pos) float64 nan nan ... nan
    ts_d_temperature        (probe, tilt, ts_radial_pos) float64 nan nan ... nan
    ts_temp_max             (probe, tilt) float64 6.935 6.307 ... 6.12 6.318
    ts_dens_max             (probe, tilt) float64 8.665e+19 ... 1.084e+20
    ts_fl                   (probe, tilt) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    shot_source_current     (probe, tilt) float64 135.3 135.4 ... 135.5 135.4
    shot_b_field            (probe, tilt) float64 1.5 1.5 1.5 ... 1.5 1.5 1.5
    shot_tilt               (probe, tilt) float64 0.007223 1.003 ... 8.995 9.978
    shot_target_pos         (probe, tilt) float64 -0.127 -0.127 ... -0.127
    shot_target_rot         (probe, tilt) float64 -89.94 -89.94 ... -89.94
    shot_target_pumpspeed   (probe, tilt) float64 82.0 82.0 82.0 ... 82.0 82.0
    shot_source_pumpspeed   (probe, tilt) float64 82.0 82.0 82.0 ... 82.0 82.0
    shot_heating_pumpspeed  (probe, tilt) float64 82.0 82.0 82.0 ... 82.0 82.0
    shot_source_pos         (probe, tilt) float64 1.2 1.2 1.2 ... 1.2 1.2 1.2
    shot_deuterium_gf       (probe, tilt) float64 0.007198 0.007138 ... 0.003577
    shot_hydrogen_gf        (probe, tilt) float64 9.32 9.332 ... 9.329 9.329
    shot_helium_gf          (probe, tilt) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    shot_hydrogen_puffed    (probe, tilt) float64 -0.01609 -0.02118 ... -0.01433
    shot_helium_puffed      (probe, tilt) float64 -0.04995 -0.04969 ... -0.0494
    adc_timestamp           (probe, tilt) float64 6.7e+18 6.7e+18 ... 6.7e+18
    adc_4_coax              (probe, tilt) float64 2.0 2.0 2.0 ... 2.0 2.0 2.0
    adc_4_shunt_resistance  (probe, tilt) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    adc_5_coax              (probe, tilt) float64 3.0 3.0 3.0 ... 3.0 3.0 3.0
    adc_5_shunt_resistance  (probe, tilt) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    adc_calibration_fl      (probe, tilt) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    shot_fl                 (probe, tilt) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    acquisition_length      (probe, tilt) float64 10.0 10.0 10.0 ... 10.0 10.0
    adc_end_timestamp       (probe, tilt) float64 6.7e+18 6.7e+18 ... 6.7e+18
    adc_freqs               (probe, tilt) float64 1e+06 1e+06 ... 1e+06 1e+06
    B                       (probe, tilt) float64 1.5 1.5 1.5 ... 1.5 1.5 1.5
    ts_temp                 (probe, tilt) object 6.935256092820191 ... 6.239160019784664
    ts_dens                 (probe, tilt) object 8.593245441413559e+19 ... 1.0805246452610525e+20
    fit_success_fl          (probe, tilt) object True True True ... True True
    temp                    (probe, tilt) float64 0.6929 1.175 ... 2.355 2.319
    d_temp                  (probe, tilt) float64 0.0571 0.03632 ... 0.07272
    isat                    (probe, tilt) float64 0.1556 0.1642 ... 0.3377
    d_isat                  (probe, tilt) float64 0.001636 ... 0.001735
    a                       (probe, tilt) float64 0.01278 0.01717 ... 0.

In [30]:
ts_temp = angle_scan_ds.ts_temperature.mean(['probe', 'tilt'])
ts_d_temp = angle_scan_ds.ts_temperature.std(['probe', 'tilt'])
ts_dens = angle_scan_ds.ts_density.mean(['probe', 'tilt'])
ts_d_dens = angle_scan_ds.ts_density.std(['probe', 'tilt'])

probe_pos_uncertainty = 1.5
ts_probe_temps = {}
ts_probe_denss = {}
for probe, probe_pos in magnum_probes.probe_position.items():
    probe_pos_limits = [probe_pos, probe_pos - probe_pos_uncertainty, probe_pos + probe_pos_uncertainty]
    ts_probe_temps[probe.upper()] = ts_temp.interp(ts_radial_pos=probe_pos_limits).values
    ts_probe_denss[probe.upper()] = ts_dens.interp(ts_radial_pos=probe_pos_limits).values


/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/xarray/core/nanops.py:142: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis=axis, dtype=dtype)
/home/jleland/coding/environments/python3.6/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1666: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


### Plotting methods

In [31]:
# The default colours from pyplot
tableau_palette = ['tab:blue', 'tab:red', 'tab:orange', 'tab:green', 'tab:purple',]

In [32]:
# phi_temp_fl = True
phi_temp_fl = False
if phi_temp_fl:
    ds_combined['d_temp_phi'] = np.abs(ds_combined['temp_phi'] - ds_combined['temp_fit_phi'])

In [33]:
fig, ax = plt.subplots(2, 3, sharex=True, constrained_layout=True)

fontsize = 18
labels = []
handles = []

for i, probe in enumerate(ds_stacked.probe.values):
    if probe not in ('S', 'L', 'B', 'L_3P'):
        continue
        
    probe_label = probe[0]
        
    ds_probe = ds_combined.sel(probe=probe)
        
    probe_dims = magnum_probes[probe_label]
        
    dummy_tilt = np.linspace(0.00001, 11.0, 1100)
#     dummy_tilt = np.linspace(89.99, 78.99, 1101)
    calced_a = lp.calc_sheath_expansion_coeff(
        ts_probe_temps[probe_label][0],
        ts_probe_denss[probe_label][0],
        probe_dims.get_2d_probe_length(),
        probe_dims.g, 
        np.radians(dummy_tilt),
        c_1=0.9,
        c_2=0.6,
    )
    calced_new_a = lp.calc_new_sheath_expansion_param(
        ts_probe_temps[probe_label][0],
        ts_probe_denss[probe_label][0],
        probe_dims.get_2d_probe_length(),
        probe_dims.g, 
        np.radians(dummy_tilt),
        probe_dims.d_perp,
        0.0 if not probe_dims.is_angled() else probe_dims.theta_p, 
        c_1=0.9,
        c_2=0.6,
    )
    
    handle = ax[0][0].errorbar('tilt', 'temp', yerr='d_temp', data=ds_probe, label=probe, fmt='.', color=tableau_palette[i])
    labels.append(probe)
    handles.append(handle)
    if phi_temp_fl:
        label = r'$T_e$ from $\phi$ on {}'.format(probe)
        handle = ax[0][0].errorbar('tilt', 'temp_phi', yerr='d_temp_phi', data=ds_probe, mfc='none', 
                                   label=label, linestyle='none', marker='^', color=tableau_palette[i])
        labels.append(label)
        handles.append(handle)
    
    ax[0][1].errorbar('tilt', 'v_f', yerr='d_v_f', data=ds_probe, fmt='.', color=tableau_palette[i])
    ax[0][2].plot('tilt', 'reduced_chi2', '.', data=ds_probe, color=tableau_palette[i])
    
    
    
    ax[1][0].errorbar('tilt', 'dens', yerr='d_dens', data=ds_probe, fmt='.', color=tableau_palette[i])
    ax[1][1].errorbar('tilt', 'isat', yerr='d_isat', data=ds_probe, fmt='.', color=tableau_palette[i])
    ax[1][2].errorbar('tilt', 'a', yerr='d_a', data=ds_probe, fmt='.', color=tableau_palette[i])
    calced_a_label = r'$a_{{{probe}, calc}}$'.format(probe=probe)
    handle = ax[1][2].errorbar(dummy_tilt, calced_a, label=calced_a_label, fmt='-', color=tableau_palette[i], 
                               linewidth=0.8, alpha=0.6)
    labels.append(calced_a_label)
    handles.append(handle)
    
    handle = ax[1][2].errorbar(dummy_tilt, calced_new_a, label=calced_a_label, fmt=':', color=tableau_palette[i], 
                               linewidth=0.8, alpha=0.6)
    
    # Plot the ts values at the probe positions using interpolaton
    handle = ax[0][0].axhline(y=ts_probe_temps[probe_label][0], color=tableau_palette[i], linewidth=1, 
                                    linestyle='dashed', label=f'TS at {probe}')
    ax[1][0].axhline(y=ts_probe_denss[probe_label][0], color=tableau_palette[i], linewidth=1, 
                  linestyle='dashed')
    
    labels.append(f'TS at {probe}')
    handles.append(handle)    

       
ax[0,0].set_ylabel('T / eV', fontsize=fontsize)
ax[0,1].set_ylabel(r'$V_f$ (V)', fontsize=fontsize)
ax[0,2].set_ylabel(r'$\chi^2_{\nu}$', fontsize=fontsize)
ax[0,2].set_yscale('log')
ax[0,2].axhline(y=1.0, **c.AX_LINE_DEFAULTS)


ax[1,0].set_ylabel(r'$n_e$ / m$^{-3}$', fontsize=fontsize)
ax[1,0].set_yscale('log')
ax[1,1].set_ylabel(r'$I_{sat}$ / A', fontsize=fontsize)
ax[1,2].set_ylabel(r'$a$', fontsize=fontsize)
ax[1,0].set_xlabel(r'Tilt / $^{\circ}$', fontsize=fontsize)
ax[1,1].set_xlabel(r'Tilt / $^{\circ}$', fontsize=fontsize)
ax[1,2].set_xlabel(r'Tilt / $^{\circ}$', fontsize=fontsize)

ax[1,2].set_yscale('log')
ax[1,2].set_xlim(-0.5, 10.5)
# ax[1,2].set_xlim(91.5, 79.5)
ax[1,2].set_ylim(2e-3, 1e0)

# for col in ax:
#     for axis in col:
#         axis.legend()
        
# fig.tight_layout(pad=0.2, w_pad=-3.5)
fig.set_constrained_layout_pads(w_pad=4./72., h_pad=4./72., hspace=0.0, wspace=3.0)
fig.legend(handles, labels, loc='right')

/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:749: UserWarning: constrained_layout not applied.  At least one axes collapsed to zero width or height.
  func(*args)
/home/jleland/.pyenv/versions/3.6.9/lib/python3.6/tkinter/__init__.py:1705: UserWarning: constrained_layout not applied.  At least one axes collapsed to zero width or height.
  return self.func(*args)


In [49]:
fig, ax = plt.subplots(4, sharex=True)

for probe in ds_stacked.probe.values:
    if probe not in ('S', 'L', 'B'):
        continue
    ax[0].errorbar('tilt', 'temp', yerr='d_temp', data=ds_stacked.sel(probe=probe), label=probe, fmt='.')
    ax[1].errorbar('tilt', 'isat', yerr='d_isat', data=ds_stacked.sel(probe=probe), label=probe, fmt='.')
    ax[2].errorbar('tilt', 'dens', yerr='d_dens', data=ds_stacked.sel(probe=probe), label=probe, fmt='.')
    ax[3].plot('tilt', 'reduced_chi2', data=ds_stacked.sel(probe=probe), label=probe)

ax[0].set_ylabel('T / eV')
ax[1].set_ylabel(r'$I_{sat}$ / A')
ax[2].set_ylabel(r'$n_e$ / m$^{-3}$')
ax[3].set_ylabel(r'$\chi^2_{\nu}$')
ax[3].set_yscale('log')
ax[3].axhline(y=1.0, **c.AX_LINE_DEFAULTS)

ax[0].legend()
ax[1].legend()

In [27]:
fig, ax = plt.subplots(3, sharex=True)

for i, probe in enumerate(ds_stacked.probe.values):
#     if probe == 'R' or probe == 'B':
    if probe == 'R':
        continue
    ax[0].errorbar('tilt', 'temp', yerr='d_temp', data=ds_stacked.sel(probe=probe), label=probe)
    ax[1].errorbar('tilt', 'dens', yerr='d_dens', data=ds_stacked.sel(probe=probe), label=probe)
    ax[2].plot('tilt', 'reduced_chi2', data=ds_stacked.sel(probe=probe), label=probe)
    
    # Plot the ts values at the probe positions using interpolaton
    ax[0].axhline(y=ts_probe_temps[probe][0], color=tableau_palette[i], 
                  linewidth=1, linestyle='dashed', label=f'TS-{probe}')
#     ax[0].axhspan(ts_probe_temps[probe][1], ts_probe_temps[probe][2], color=tableau_palette[i], alpha=0.25)
    ax[1].axhline(y=ts_probe_denss[probe][0], color=tableau_palette[i], 
                  linewidth=1, linestyle='dashed', label=f'TS-{probe}')
#     ax[1].axhspan(ts_probe_denss[probe][1], ts_probe_denss[probe][2], color=tableau_palette[i], alpha=0.25)

ax[0].set_ylabel('T / eV')
ax[1].set_ylabel(r'$n_e$ / m$^{-3}$')
ax[2].set_ylabel(r'$\chi^2_{\nu}$')
ax[2].set_xlabel(r'probe tilt / $^{\circ}$')


# Plot the reduced chi^2 on a log plot and indicate where 1 is
ax[2].axhline(y=1.0, **c.AX_LINE_DEFAULTS)
# ax[2].set_ylim(0, 3)
ax[2].set_yscale('log')

# ax[0].legend()
# ax[1].legend()
ax[2].legend()

In [56]:
fig, ax = plt.subplots(2)

for probe in ds_stacked.probe.values:
    ax[0].errorbar('tilt', 'isat', yerr='d_isat', data=ds_stacked.sel(probe=probe), label=probe)
    ax[1].plot('tilt', 'reduced_chi2', data=ds_stacked.sel(probe=probe), label=probe)
    
ax[0].legend()
ax[1].legend()

In [76]:


fig, ax = plt.subplots(2, sharex=True)
ax[0].set_ylabel('T / eV')
ax[1].set_ylabel(r'$n_e$ / m$^{-3}$')
# ax[0].errorbar(helium_anglescan_ds.ts_radial_pos, ts_temp, yerr=helium_anglescan_ds.ts_d_temperature.mean('shot_number'), color='silver', ecolor='silver')
ax[0].errorbar(angle_scan_ds.ts_radial_pos, ts_temp, yerr=ts_d_temp, color='k')
ax[1].errorbar(angle_scan_ds.ts_radial_pos, ts_dens, yerr=ts_d_dens, color='k')


for probe, probe_pos in magnum_probes.probe_position.items():
#         ax[i].axvline(x=probe_pos, color='black', linewidth=1, linestyle='dashed')
    if probe == 'r':
        continue
    ds_probe = ds_stacked.sel(probe=probe.upper())
    ax[0].errorbar(probe_pos * np.ones_like(ds_probe['tilt'].values), ds_probe['temp'].values, 
                   yerr=ds_probe['d_temp'], xerr=2.5, 
                   linestyle='none', label=probe)
    ax[1].errorbar(probe_pos * np.ones_like(ds_probe['tilt'].values), ds_probe['dens'].values, 
                   yerr=ds_probe['d_dens'], xerr=2.5, 
                   linestyle='none', label=probe)
        
plt.legend()
plt.show()
fig.suptitle(f'Thomson Scattering profiles with probe fits overlaid')

Text(0.5, 0.98, 'Thomson Scattering profiles with probe fits overlaid')

In [78]:
fig, ax = plt.subplots()
markers = ['.', 'x', '+', '^', '*', 'o']
colors = ['red', 'blue', 'green', 'orange', 'purple', 'black']

coarse_s_ds = ds_combined.sel(probe='S').isel(tilt=slice(0,11,2))
for i, tilt in enumerate(coarse_s_ds.tilt.values):
    ds = coarse_s_ds.sel(tilt=tilt)
#     coarse_s_ds.set_coords('voltage')['current'].plot.line(hue='tilt', x='voltage', ax=ax)
    ax.plot(ds['voltage'].values, -ds['current'].values, marker=markers[i], mfc='none', color=colors[i], linestyle='none', label=r'${:.2g}^{{\circ}}$'.format(tilt))
    ax.set_ylabel('Probe Current [A]')
    ax.set_xlabel('Probe Voltage [V]')
    ax.axhline(y=0, **c.AX_LINE_DEFAULTS)
    ax.legend(title='Probe Tilt')


In [80]:
fig, ax = plt.subplots(2, sharex=True)

angle_offset = 0

ds = ds_combined.sel(probe='S').sortby('tilt', ascending=False)
ds['normal_tilt'] -= angle_offset
ax[0].errorbar('normal_tilt', 'isat', yerr='d_isat', data=ds, label='S', fmt='xk', ms=1.5)
ax[0].invert_xaxis()
ax[0].set_ylim(0, 0.81)
ax[0].set_xlim(91, 79 - angle_offset)

theta = np.linspace(90, 78 - angle_offset ,1000)
cos_theta = np.linspace(0, np.cos(np.radians(78 - angle_offset)), 1000)
cos_ax = ax[0].twinx()
cos_ax.plot(theta, cos_theta, **c.AX_LINE_DEFAULTS)
cos_ax.plot(theta+22, np.flip(cos_theta), **c.AX_LINE_DEFAULTS)
cos_ax.set_ylim(0, 0.35)

ax[1].errorbar('normal_tilt', 'dens', yerr='d_isat', data=ds, label='S', fmt='xk', ms=1.5)
ax[1].set_ylim(0)
# ax[1].legend()

(0, 4.141379853346467e+19)

### Testing area

In [13]:
super_title = 'H axial scan @ 0.8T, tilt=10'
axial_scan_indices = [132,148,149,150,151,152,153,154,155,156,157,158,159,160,161,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189]

In [14]:
axial_scan_ds = mgut.get_dataset_from_indices(axial_scan_indices, anglescan_fl=False, 
                                              path_to_analysed_datasets=path_to_analysed_datasets, 
                                              average_direction_fl=False)

In [24]:
angle_scan_ds

<xarray.Dataset>
Dimensions:                 (probe: 4, tilt: 11, time: 50, ts_radial_pos: 46)
Coordinates:
  * ts_radial_pos           (ts_radial_pos) float64 -36.47 -34.8 ... 37.14 38.82
  * tilt                    (tilt) float64 0.0 1.0 2.0 3.0 ... 7.0 8.0 9.0 10.0
  * probe                   (probe) object 'S' 'L' 'R' 'B'
Dimensions without coordinates: time
Data variables:
    voltage                 (tilt, probe, time) float64 -105.4 -103.3 ... -89.21
    current                 (tilt, probe, time) float64 -0.1868 ... -0.532
    shot_time               (tilt, probe, time) float64 4.373 4.373 ... 4.758
    start_time              (tilt, probe) float64 4.37 4.37 ... 4.756 4.756
    d_current               (tilt, probe, time) float64 0.0006304 ... 0.004755
    std_current             (tilt, probe, time) float64 0.02315 ... 0.1843
    ts_density              (probe, tilt, ts_radial_pos) float64 nan nan ... nan
    ts_temperature          (probe, tilt, ts_radial_pos) float64 nan nan ... nan
    ts_d_density            (probe, tilt, ts_radial_pos) float64 nan nan ... nan
    ts_d_temperature        (probe, tilt, ts_radial_pos) float64 nan nan ... nan
    ts_temp_max             (probe, tilt) float64 7.802 7.882 ... 8.352 8.131
    ts_dens_max             (probe, tilt) float64 7.128e+19 ... 8.784e+19
    ts_fl                   (probe, tilt) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    shot_source_current     (probe, tilt) float64 99.91 100.1 ... 100.0 99.84
    shot_b_field            (probe, tilt) float64 1.2 1.2 1.2 ... 1.2 1.2 1.2
    shot_tilt               (probe, tilt) float64 0.01012 1.004 ... 9.0 9.988
    shot_target_pos         (probe, tilt) float64 -0.2 -0.2 -0.2 ... -0.2 -0.2
    shot_target_rot         (probe, tilt) float64 -89.94 -89.94 ... -89.94
    shot_target_pumpspeed   (probe, tilt) float64 82.0 82.0 82.0 ... 82.0 82.0
    shot_source_pumpspeed   (probe, tilt) float64 82.0 82.0 82.0 ... 82.0 82.0
    shot_heating_pumpspeed  (probe, tilt) float64 82.0 82.0 82.0 ... 82.0 82.0
    shot_source_pos         (probe, tilt) float64 1.35 1.35 1.35 ... 1.35 1.35
    shot_deuterium_gf       (probe, tilt) float64 0.01172 0.005637 ... 0.007478
    shot_hydrogen_gf        (probe, tilt) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    shot_helium_gf          (probe, tilt) float64 8.443 8.439 ... 8.443 8.441
    shot_hydrogen_puffed    (probe, tilt) float64 -0.01761 -0.0164 ... -0.02437
    shot_helium_puffed      (probe, tilt) float64 -0.04923 -0.04622 ... -0.04716
    adc_timestamp           (probe, tilt) float64 6.699e+18 ... 6.699e+18
    adc_4_coax              (probe, tilt) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    adc_4_shunt_resistance  (probe, tilt) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    adc_5_coax              (probe, tilt) float64 2.0 2.0 2.0 ... 2.0 2.0 2.0
    adc_5_shunt_resistance  (probe, tilt) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    adc_calibration_fl      (probe, tilt) float64 0.0 0.0 0.0 ... 0.0 0.0 0.0
    shot_fl                 (probe, tilt) float64 1.0 1.0 1.0 ... 1.0 1.0 1.0
    acquisition_length      (probe, tilt) float64 8.0 8.0 8.0 ... 8.0 8.0 11.5
    adc_end_timestamp       (probe, tilt) float64 6.699e+18 ... 6.699e+18
    adc_freqs               (probe, tilt) float64 1e+06 1e+06 ... 1e+06 1e+06

In [16]:
axial_scan_vars = set(axial_scan_ds.variables.keys())

In [17]:
droppable_vars = axial_scan_vars - angles_scan_vars - {'shot_number'}
droppable_vars

{'adc_4_probe',
 'adc_5_probe',
 'adc_calibration_index',
 'adc_end_time',
 'adc_filename',
 'adc_folder',
 'adc_index',
 'adc_time',
 'shot_end_time',
 'sweep_time',
 'ts_number',
 'ts_time',
 'ts_timestamp'}

In [31]:
# axial_scan_ds
ds = angle_scan_ds.sel(probe='L', tilt=5.0)

In [36]:
iv_data = iv.IVData.from_dataset(ds)
iv_data['I'] = -iv_data['I']

In [37]:
iv_data.plot()

<ErrorbarContainer object of 3 artists>

In [38]:
simp_fitter = fts.SimpleIVFitter()
simp_fitter.fit_iv_data(iv_data).plot()

In [70]:
import scipy.optimize as opt

initial_vals = simp_fitter.default_values
bounds = simp_fitter.default_bounds

fit_vals, fit_cov = opt.curve_fit(simp_fitter.fit_function, iv_data['V'], iv_data['I'], p0=[1.0, 10.0, -40]) #, p0=initial_vals) #, bounds=bounds, sigma=iv_data['sigma'])
print(fit_vals)
print(initial_vals)

[ 4.27562016e-03  1.16470767e+01 -5.65021344e+01]
(30.0, 1, -1)


In [71]:
fig, ax = plt.subplots()

iv_data.plot(ax=ax)
ax.plot(iv_data['V'], iv_fit_function(iv_data['V'], *[0.004, 10, -40]), label='manual')
ax.plot(iv_data['V'], simp_fitter.fit_function(iv_data['V'], *fit_vals), label='fit')
ax.legend()

In [60]:
def iv_fit_function(v, *parameters):
    I_0 = parameters[0]
    T_e = parameters[1]
    v_f = parameters[2]
    V = (v_f - v) / T_e

    return I_0 * (1 - np.exp(-V))